In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#import scipy.stats as stats
#import statsmodels.api as sm
import re
import plotly.express as px
import requests
from bs4 import BeautifulSoup as BS
from io import StringIO
from shapely.geometry import Point
#from scipy.spatial import distance
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import math
import geopy.distance
from tqdm.notebook import tqdm
%matplotlib inline

In [18]:
quakes = pd.read_csv('../data/quakes_cities.csv')

In [19]:
quakes

,Unnamed: 0,Unnamed: 0.1,origin,country,lat,long,depth_km,magnitude,secondary_effects,pde_shaking_deaths,...,year,index,gdp_per_capita,density,geometry,coordinates,distance,city_coord,city,population
0,0,0,1900-05-11 17:23,Japan,38.700,141.100,5.0,7.0 MJMA,NaN,NaN,...,1900,NaN,NaN,NaN,POINT (141.1 38.7),"('38.700', '141.100')",7.618754969874188 km,"(38.7303, 141.0214)",Kurihara,65313.0
1,1,1,1900-07-12 06:25,Turkey,40.300,43.100,NaN,5.9 Muk,NaN,NaN,...,1900,NaN,NaN,NaN,POINT (43.1 40.3),"('40.300', '43.100')",34.08423281320913 km,"(40.6069, 43.0931)",Kars,112260.0
2,2,2,1900-10-29 09:11,Venezuela,11.000,-66.000,0.0,7.7 Mw,NaN,NaN,...,1900,NaN,NaN,NaN,POINT (-66 11),"('11.000', '-66.000')",82.78539644809187 km,"(10.4739, -66.5383)",Guarenas,208663.0
3,3,3,1901-02-15 00:00,China,26.000,100.100,0.0,6.5 Ms,NaN,NaN,...,1901,NaN,NaN,NaN,POINT (100.1 26),"('26.000', '100.100')",33.68040241938257 km,"(25.7003, 100.1564)",Dali,652045.0
4,4,4,1901-03-31 07:11,Bulgaria,43.400,28.700,NaN,6.4 Muk,NaN,NaN,...,1901,NaN,NaN,NaN,POINT (28.7 43.4),"('43.400', '28.700')",67.03085363604131 km,"(43.2078, 27.9169)",Varna,369162.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319,1319,1328,2010-02-27 06:34,Chile,-35.846,-72.719,35.0,8.8 Mw,T,NaN,...,2010,1105.0,19363.220303,23.604875,POINT (-72.71899999999999 -35.846),"('-35.846', '-72.719')",100.94065331242729 km,"(-36.6067, -72.1033)",Chillán,164270.0
1320,1320,1334,2011-03-11 14:46,Japan,38.322,142.369,24.4,9.0 Mw,T,NaN,...,2011,2960.0,35774.696713,301.427706,POINT (142.369 38.322),"('38.322', '142.369')",93.62285222964798 km,"(38.4281, 141.3061)",Ishizaki,141293.0
1321,1321,1336,2011-04-07 14:32,Japan,38.200,140.000,66.0,7.1 Mw,NaN,NaN,...,2011,2960.0,35774.696713,301.427706,POINT (140 38.2),"('38.2', '140.0')",29.707905325922752 km,"(38.25, 140.3333)",Yamagata,249327.0
1322,1322,1337,2011-09-18 12:40,India,27.723,88.064,19.7,6.9 Mw,L,NaN,...,2011,2729.0,4624.561404,387.978018,POINT (88.06399999999999 27.723),"('27.723', '88.064')",70.0956631052954 km,"(27.33, 88.62)",Gangtok,100286.0


In [20]:
quake_deaths = quakes[quakes['deaths'].notna()].groupby(
    'country')[['country', 'deaths']].agg(['count', 'sum', np.mean, 'median', 'max'])
quake_deaths

deaths                                         
                   count      sum          mean   median      max
country                                                          
Afghanistan           28   9338.0    333.500000      8.0   4000.0
Albania                7    244.0     34.857143     25.0    120.0
Algeria               19   9330.0    491.052632      9.0   5000.0
Argentina             11   8584.0    780.363636      8.0   8000.0
Armenia                2  25390.0  12695.000000  12695.0  25000.0
...                  ...      ...           ...      ...      ...
Venezuela             10    442.0     44.200000      4.0    240.0
Vietnam                1      1.0      1.000000      1.0      1.0
Wallis and Futuna      1      5.0      5.000000      5.0      5.0
Yemen                  3   4011.0   1337.000000   1200.0   2800.0
Yugoslavia             2     47.0     23.500000     23.5     35.0

[99 rows x 5 columns]

In [21]:
quake_deaths.columns = ['count', 'sum', 'mean', 'median', 'max']

In [22]:
quake_deaths.sort_values('count', ascending=False).head(10)

,count,sum,mean,median,max
country,,,,,
China,145,1138388.0,7850.951724,12.0,655237.0
Iran,90,152677.0,1696.411111,21.0,45000.0
Turkey,89,90836.0,1020.629213,23.0,32968.0
Indonesia,86,342112.0,3978.046512,15.5,297200.0
Japan,76,166997.0,2197.328947,8.5,142807.0
Peru,52,73941.0,1421.942308,10.5,70000.0
Taiwan,48,7954.0,165.708333,5.0,3276.0
Mexico,42,12706.0,302.523810,19.0,9500.0
Chile,39,39461.0,1011.820513,3.0,28000.0


In [23]:
quake_deaths.sort_values('sum', ascending=False).head(10)

,count,sum,mean,median,max
country,,,,,
China,145,1138388.0,7850.951724,12.0,655237.0
Indonesia,86,342112.0,3978.046512,15.5,297200.0
Japan,76,166997.0,2197.328947,8.5,142807.0
Pakistan,23,153910.0,6691.739130,19.0,87351.0
Iran,90,152677.0,1696.411111,21.0,45000.0
Italy,33,121423.0,3679.484848,5.0,82000.0
Turkey,89,90836.0,1020.629213,23.0,32968.0
Peru,52,73941.0,1421.942308,10.5,70000.0
India,23,52354.0,2276.260870,15.0,20023.0


In [24]:
quake_deaths.sort_values('mean', ascending=False).head(10)

,count,sum,mean,median,max
country,,,,,
Armenia,2,25390.0,12695.000000,12695.0,25000.0
China,145,1138388.0,7850.951724,12.0,655237.0
Pakistan,23,153910.0,6691.739130,19.0,87351.0
Turkmenistan,4,23468.0,5867.000000,1828.5,19800.0
Indonesia,86,342112.0,3978.046512,15.5,297200.0
Italy,33,121423.0,3679.484848,5.0,82000.0
Morocco,4,13735.0,3433.750000,316.5,13100.0
Tajikistan,8,27324.0,3415.500000,20.5,15000.0
Nepal,4,12418.0,3104.500000,819.0,10700.0


In [25]:
quake_deaths.sort_values('median', ascending=False).head(10)

,count,sum,mean,median,max
country,,,,,
Armenia,2,25390.0,12695.00,12695.0,25000.0
Turkmenistan,4,23468.0,5867.00,1828.5,19800.0
Yemen,3,4011.0,1337.00,1200.0,2800.0
Nepal,4,12418.0,3104.50,819.0,10700.0
Guinea,1,643.0,643.00,643.0,643.0
Jamaica,2,1001.0,500.50,500.5,1000.0
Morocco,4,13735.0,3433.75,316.5,13100.0
Palestine,2,381.0,190.50,190.5,361.0
Montenegro,1,156.0,156.00,156.0,156.0


In [26]:
quake_deaths.sort_values('max', ascending=False).head(10)

,count,sum,mean,median,max
country,,,,,
China,145,1138388.0,7850.951724,12.0,655237.0
Indonesia,86,342112.0,3978.046512,15.5,297200.0
Japan,76,166997.0,2197.328947,8.5,142807.0
Pakistan,23,153910.0,6691.739130,19.0,87351.0
Italy,33,121423.0,3679.484848,5.0,82000.0
Peru,52,73941.0,1421.942308,10.5,70000.0
Iran,90,152677.0,1696.411111,21.0,45000.0
Turkey,89,90836.0,1020.629213,23.0,32968.0
Chile,39,39461.0,1011.820513,3.0,28000.0


In [27]:
fig = px.scatter(quakes[quakes['magnitude.1'].notna()], x="density", y="deaths", size='magnitude.1',
           hover_name="country", size_max=15, log_x=True, log_y=True)
fig.update_layout(
    title="Earthquake Deaths by Country Population Density",
    xaxis_title="Population Density by Country",
    yaxis_title="Number of Deaths"
    )
fig.show()
#fig.write_html('mp_03_accrepresentation.html')

In [28]:
fig = px.scatter(quakes[quakes['magnitude.1'].notna()], x="gdp_per_capita", y="deaths", size='magnitude.1',
           hover_name="country", size_max=20, log_x=True, log_y=True)
fig.update_layout(
    title="Earthquake Deaths by Country GDP per Capita",
    xaxis_title="Country GDP per Capita",
    yaxis_title="Number of Deaths"
    )
fig.show()
#fig.write_html('mp_03_accrepresentation.html')

## MAPPING

In [29]:
center = [0, 0]
quake_map = folium.Map(location = center, zoom_start=2)

In [30]:
for row_index, row_values in quakes.iterrows():
    loc = [row_values['lat'], row_values['long']]
    pop = str(row_values['origin'])
    marker = folium.Marker(
        location = loc,
        popup = pop)
    
    marker.add_to(quake_map)

quake_map

In [31]:
cluster_quake_map = folium.Map(location = center, zoom_start=2)
marker_cluster = MarkerCluster().add_to(cluster_quake_map)

for row_index, row_values in quakes.iterrows():
    loc = [row_values['lat'], row_values['long']]
    pop = str(row_values['origin'])
    marker = folium.Marker(
        location = loc, 
        popup = pop)
    
    marker.add_to(marker_cluster)
    
cluster_quake_map

## Do the magnitudes of earthquakes seem to follow the distribution described by Nate Silver. That is, one higher magnitude is ten times less likely to occur?

In [54]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'

In [55]:
payload = {'format' :'csv',
          'starttime':'1900-01-01',
          'minlatitude':'33.203247',
          'maxlatitude':'38.821584',
          'minlongitude':'-93.258663',
          'maxlongitude':'-80.398366',
          'limit': [1,20000]}

In [56]:
r = requests.get(url=url, params=payload)

In [57]:
usgs_data = pd.read_csv(StringIO(r.text))

In [63]:
usgs_data

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2020-10-27T10:35:29.420Z,36.543833,-81.050667,0.16,-0.43,md,5.0,309.0,0.04557,0.33,...,2020-10-27T17:44:15.300Z,"7 km NE of Sparta, North Carolina",earthquake,2.65,25.90,0.099,4.0,reviewed,se,se
1,2020-10-27T00:02:18.930Z,36.439333,-90.841500,7.92,2.53,md,11.0,136.0,0.44680,0.25,...,2020-10-27T12:26:05.010Z,"5 km ENE of Maynard, Arkansas",earthquake,1.34,3.68,0.098,6.0,reviewed,nm,nm
2,2020-10-26T08:35:34.140Z,36.253167,-89.417500,6.88,2.03,md,25.0,60.0,0.02304,0.09,...,2020-10-26T13:59:48.800Z,"6 km E of Ridgely, Tennessee",earthquake,0.18,0.37,0.092,20.0,reviewed,nm,nm
3,2020-10-26T07:46:45.230Z,36.425333,-90.824833,9.10,2.44,md,16.0,92.0,0.44330,0.18,...,2020-10-26T15:16:10.010Z,"6 km E of Maynard, Arkansas",earthquake,0.52,1.70,0.138,12.0,reviewed,nm,nm
4,2020-10-26T04:08:11.800Z,35.615167,-89.690667,1.51,1.86,md,10.0,233.0,0.14130,0.28,...,2020-10-26T14:58:33.790Z,"6 km ENE of Garland, Tennessee",earthquake,0.94,1.50,0.051,9.0,reviewed,nm,nm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14062,1903-11-04T19:14:00.000Z,36.500000,-89.800000,NaN,5.10,fa,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,Missouri,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,sc
14063,1903-11-04T18:18:00.000Z,36.500000,-89.500000,NaN,4.60,fa,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,Tennessee,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,sc
14064,1903-02-09T00:21:00.000Z,37.800000,-89.300000,NaN,4.90,fa,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,Illinois,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,sc
14065,1902-01-24T10:48:00.000Z,38.600000,-90.300000,NaN,4.50,fa,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,Missouri,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,sc


In [64]:
TN_Earthquakes = usgs_data[usgs_data['place'].str.contains('Tennessee')]

In [66]:
TN_Earthquakes

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
2,2020-10-26T08:35:34.140Z,36.253167,-89.417500,6.88,2.03,md,25.0,60.0,0.02304,0.09,...,2020-10-26T13:59:48.800Z,"6 km E of Ridgely, Tennessee",earthquake,0.18,0.37,0.092,20.0,reviewed,nm,nm
4,2020-10-26T04:08:11.800Z,35.615167,-89.690667,1.51,1.86,md,10.0,233.0,0.14130,0.28,...,2020-10-26T14:58:33.790Z,"6 km ENE of Garland, Tennessee",earthquake,0.94,1.50,0.051,9.0,reviewed,nm,nm
7,2020-10-25T17:18:57.880Z,35.666833,-84.024667,18.54,2.45,md,23.0,80.0,0.14050,0.18,...,2020-10-27T15:36:56.862Z,"11 km SSW of Maryville, Tennessee",earthquake,0.31,0.59,0.089,17.0,reviewed,se,se
11,2020-10-23T04:09:18.260Z,36.332833,-89.484667,4.93,1.65,md,24.0,47.0,0.01857,0.03,...,2020-10-23T16:52:09.410Z,"5 km SSW of Tiptonville, Tennessee",earthquake,0.33,0.27,0.068,14.0,reviewed,nm,nm
17,2020-10-21T03:00:08.800Z,36.313667,-89.536167,9.50,1.23,md,17.0,65.0,0.02693,0.06,...,2020-10-21T13:18:24.430Z,"7 km NW of Ridgely, Tennessee",earthquake,0.68,0.51,0.032,10.0,reviewed,nm,nm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14032,1952-07-16T23:48:10.000Z,36.200000,-89.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,Tennessee,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,ushis
14040,1928-11-03T04:02:50.200Z,36.112000,-82.828000,5.00,4.50,lg,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,eastern Tennessee,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,dg
14050,1918-10-16T02:15:00.000Z,36.000000,-89.200000,NaN,4.50,fa,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,Tennessee,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,sg
14057,1913-03-28T21:50:00.000Z,36.200000,-83.700000,NaN,4.10,fa,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,eastern Tennessee,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,sc


In [67]:
fig = px.scatter(
    usgs_data[usgs_data['mag'].notna()], 
    x=usgs_data[usgs_data['mag'].notna()]['mag'].unique(), 
    y=usgs_data['mag'].value_counts(),
    log_x=True, 
    log_y=True
)
fig.update_layout(
    title="Earthquakes by Magnitude",
    xaxis_title="Magnitude of Earthquake",
    yaxis_title="Count of Earthquakes by Magnitude"
    )
fig.show()
#fig.write_html('mp_03_accrepresentation.html')

In [78]:
TN_Earthquakes[TN_Earthquakes['mag'].notna()]['mag'].round(1)

2        2.0
4        1.9
7        2.4
11       1.6
17       1.2
        ... 
14028    3.9
14040    4.5
14050    4.5
14057    4.1
14063    4.6
Name: mag, Length: 4766, dtype: float64

In [89]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [94]:
fig = px.scatter(
    TN_Earthquakes[TN_Earthquakes['mag'].notna()], 
    x=TN_Earthquakes[TN_Earthquakes['mag'].notna()]['mag'].round(1).unique(), 
    y=TN_Earthquakes[TN_Earthquakes['mag'].notna()]['mag'].round(1).value_counts(),
    range_x=[2,5],
    trendline='ols',
    log_x=True, 
    log_y=True
)
fig.update_layout(
    title="Earthquakes by Magnitude",
    xaxis_title="Magnitude of Earthquake",
    yaxis_title="Count of Earthquakes by Magnitude"
    )
fig.show()
#fig.write_html('mp_03_accrepresentation.html')

In [ ]:
states = gpd.read_file('tl_2019_us_statex.shx')
type(states)